Para replicar el referenciamiento de ImageNet en tu propio dataset de imágenes almacenado en Amazon S3 y ejecutar la transferencia de aprendizaje con VGG16, debes seguir varios pasos. Aquí tienes una guía general sobre cómo hacerlo:


In [2]:
# Librerías estándar
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO

# TensorFlow y Keras
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import (Conv2D, Dense, Dropout, Flatten, GlobalAveragePooling2D, MaxPooling2D, Reshape)
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

# Otras librerías
import os
import boto3
from PIL import Image
from sklearn.metrics import (accuracy_score, auc, average_precision_score, classification_report, 
                             confusion_matrix, precision_score, recall_score)
from sklearn.model_selection import train_test_split

## Configuración de AWS S3:
Asegúrate de tener acceso a tu bucket de S3 desde tu entorno de desarrollo. Puedes hacer esto configurando las credenciales de AWS adecuadamente.


In [9]:
# Importo Informacion del AccessKey.
accessKey = os.getenv("AccessKey")
secretKey = os.getenv("SecretKey")

In [10]:
# Configura tus credenciales de AWS si aún no lo has hecho
boto3.setup_default_session(aws_access_key_id=accessKey,
                           aws_secret_access_key=secretKey,
                           region_name='us-east-2')

In [11]:
# Nombre del bucket
nombre_bucket_s3 = 's3-pfa'

In [12]:
# Crea un cliente de S3
s3 = boto3.client('s3')

## Generacion de Tensores

In [13]:
input_shape = (150, 150, 3)
# Definir una función para cargar imágenes desde S3
def cargar_imagen_desde_s3(ruta, target_size=(input_shape[0], input_shape[1])):
    obj = s3.get_object(Bucket=nombre_bucket_s3, Key=ruta)
    img = Image.open(BytesIO(obj['Body'].read()))

    # Redimensionar la imagen al tamaño objetivo
    img = tf.image.resize(img, target_size)

    return img

## Preparación de datos:
Asegúrate de que tus imágenes estén almacenadas en un formato accesible desde Python, como un DataFrame de Pandas o una estructura de directorios.

In [37]:
df = pd.read_pickle('ArchivosUtiles/dataset_entrenamiento.pkl')

In [46]:
df['Animal'].value_counts()

Animal
1    3896
0    3653
Name: count, dtype: int64

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7549 entries, 0 to 1615
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Ruta    7549 non-null   object
 1   Animal  7549 non-null   int64 
 2   Imagen  7549 non-null   object
dtypes: int64(1), object(2)
memory usage: 235.9+ KB


In [103]:
df_Especie = df[(df['Especie'].notna()) & (df['Especie'] != 'nula')]

In [ ]:
df_Especie['Guanaco'] = np.where(df_Especie['Especie'] == 'guanaco', True, False)

/var/folders/6c/17qbm4nd5318hd4bcrx68wjh0000gn/T/ipykernel_3570/231559783.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Especie['Guanaco'] = np.where(df_Especie['Especie'] == 'guanaco', True, False)


In [ ]:
# Listas de categorías
aves = ('martineta', 'choique', 'passeriforme', 'tucuquere', 'gaviota', 'jote', 'chimango', 'ave s/i', 'flamenco', 'aguila mora', 'garza blanca', 'halcon', 'atahacaminos', 'bandurria', 'chingolo', 'loica', 'calandria', 'zorzal chiguanco', 'yal negro', 'torcaza', 'carancho', 'aguilucho comun', 'cormoran imperial', 'cauquen', 'pato')
roedor = ('raton', 'piche')
mam_herbivoro = ('guanaco', 'liebre', 'mara', 'oveja', 'piche')
mam_carnivoro = ('zorro gris', 'gato montes', 'peludo', 'zorrino', 'puma', 'zorro colorado', 'raton')

# Función para asignar categoría
def asignar_categoria(especie):
    if especie in aves:
        return 'Ave'
    elif especie in mam_herbivoro:
        return 'Mamifero Herbivoro'
    elif especie in mam_carnivoro:
        return 'Mamifero Carnivoro'
    else:
        return 'Otra'  # Puedes cambiar esto según tus necesidades

In [ ]:
# Aplicar la función para crear una nueva columna 'Categoria'
df_Especie['Categoria'] = df_Especie['Especie'].apply(asignar_categoria)

/var/folders/6c/17qbm4nd5318hd4bcrx68wjh0000gn/T/ipykernel_3570/2632377907.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Especie['Categoria'] = df_Especie['Especie'].apply(asignar_categoria)


In [423]:
df_Especie = df_Especie.drop_duplicates(subset='Ruta', keep=False)

In [62]:
df_Especie = df_Especie[df_Especie['Ruta']!= 'Muestreo ct sauce/CT S 2020/S16/S16 - (246).JPG'] # 2 animales

In [63]:
df_Especie.to_pickle('dataset_entrenamiento_Especie.pkl')

In [3]:
df_Especie = pd.read_pickle('ArchivosUtiles/dataset_entrenamiento_Especie.pkl')

In [5]:
df_Especie.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3559 entries, 0 to 1615
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Ruta          3559 non-null   object 
 1   Especie       2161 non-null   object 
 2   Cantidad      2158 non-null   float64
 3   Animal        2161 non-null   float64
 4   Imagen        3559 non-null   object 
 5   animal_count  0 non-null      float64
 6   Guanaco       3559 non-null   object 
 7   Categoria     2161 non-null   object 
dtypes: float64(3), object(5)
memory usage: 250.2+ KB


In [14]:
df_Especie['Imagen'] = df_Especie['Ruta'].apply(cargar_imagen_desde_s3)

In [15]:
df_Especie.to_pickle('ArchivosUtiles/dataset_entrenamiento_Especie.pkl')

In [12]:
df = pd.read_pickle('ArchivosUtiles/dataset_entrenamiento_Aislado.pkl')

## Split en Entrenamiento y Validacion

In [48]:
# Divide el DataFrame en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df[['Ruta','Imagen']], df['Animal'], test_size=0.2, random_state=42, stratify=df['Animal'])

In [50]:
train = pd.DataFrame({'Ruta': X_train['Ruta'], 'Imagen': X_train['Imagen'], 'Animal': y_train})
test = pd.DataFrame({'Ruta': X_test['Ruta'], 'Imagen': X_test['Imagen'], 'Animal': y_test})

In [51]:
train.to_pickle('ArchivosUtiles/trainingAnimal.pkl')
test.to_pickle('ArchivosUtiles/testingAnimal.pkl')

In [16]:
# Divide el DataFrame en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df_Especie[['Ruta','Imagen']], df_Especie['Guanaco'], test_size=0.2, random_state=42, stratify=df_Especie['Guanaco'])

In [17]:
train = pd.DataFrame({'Ruta': X_train['Ruta'], 'Imagen': X_train['Imagen'], 'Guanaco': y_train})
test = pd.DataFrame({'Ruta': X_test['Ruta'], 'Imagen': X_test['Imagen'], 'Guanaco': y_test})

In [18]:
train.to_pickle('trainingGuanaco.pkl')
test.to_pickle('testingGuanaco.pkl')

In [ ]:
# Divide el DataFrame en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df_Especie['Imagen'], df_Especie['Categoria'], test_size=0.2, random_state=42, stratify=df_Especie['Categoria'])

In [ ]:
train = pd.DataFrame({'Imagen': X_train, 'Categoria': y_train})
test = pd.DataFrame({'Imagen': X_test, 'Categoria': y_test})

In [ ]:
train.to_pickle('trainingCategoria.pkl')
test.to_pickle('testingCategoria.pkl')